# **What are .nii files?**
NII format is created by Neuroimaging Informatics Technology Initiative. It is commonly used to store magnetic resonance imaging (MRI) data.

This sections of the code is just to see how to open and visualize this .nii data in python.

In [ ]:
import nibabel as nib
from glob import glob
import matplotlib.pylab as plt

In [ ]:
path = '/content/BraTS20_Training_001_flair.nii'

In [ ]:
img = nib.load(path).get_fdata()
img.shape

In [ ]:
print(f"The .nii files are stored in memory as numpy's: {type(img)}.")

# **Axial slices**
The code here is to visualize the the Axial of the image.

The Axial plane is an anatomical plane that divides the body into x and y plane.

In [ ]:
num_slices = img.shape[2]

In [ ]:
num_columns = 4
num_rows = (num_slices + num_columns - 1) // num_columns


In [ ]:
# Visualize all Axial slices from the start
for i in range(num_slices):
    fig, ax = plt.subplots(figsize=(6, 6))  # Adjust figsize as needed
    ax.imshow(img[:, :, i], cmap='gray', aspect='auto', vmin=0, vmax=255)
    ax.axis('off')
    plt.show()
    print(i)

# **Coronal slices**

The code here is to visualize the the coronal of the image.

The coronal plane is an anatomical plane that divides the body into dorsal and ventral sections. It is perpendicular to the sagittal and transverse planes.

In [ ]:
# Check the shape of the MRI data
print("Shape of MRI data:", img.shape)

# Calculate the middle slice index
num_coronal_slices = img.shape[0] 

In [ ]:
# Visualize all coronal slices from the start
for i in range(num_coronal_slices):
    fig, ax = plt.subplots(figsize=(6, 6))  # Adjust figsize as needed
    ax.imshow(img[i, :, :], cmap='gray', aspect='auto', vmin=0, vmax=255)
    ax.axis('off')
    plt.show()
    print(i)

# **Sagittal slices**
The code here is to visualize the the sagittal of the image.

In [ ]:
# Check the shape of the MRI data
print("Shape of MRI data:", img.shape)

# Determine the number of slices in the second dimension (sagittal plane)
num_sagittal_slices = img.shape[1]

In [ ]:
# Visualize all sagittal slices from the start
for i in range(num_sagittal_slices):
    fig, ax = plt.subplots(figsize=(6, 6))  # Adjust figsize as needed
    ax.imshow(img[:, i, :], cmap='gray', aspect='auto', vmin=0, vmax=255)
    ax.axis('off')
    plt.show()